In [1]:
import numpy
from scipy.stats import laplace, norm, t
import scipy
import math
import numpy as np

In [2]:
VARIANCE = 2.0

normal_scale = math.sqrt(VARIANCE)
student_t_df = (2 * VARIANCE) / (VARIANCE - 1)
laplace_scale = VARIANCE / 2

HYPOTHESIS_SPACE = [norm(loc=0.0, scale=normal_scale),
                    laplace(loc=0.0, scale=laplace_scale),
                    t(df=student_t_df)]

PRIOR_PROBS = np.array([0.35, 0.25, 0.4])

In [7]:
def generate_sample(n_samples, seed=None):
    """ data generating process of the Bayesian model """
    random_state = np.random.RandomState(seed)
    hypothesis_idx = np.random.choice(3, p=PRIOR_PROBS)
    dist = HYPOTHESIS_SPACE[hypothesis_idx]
    return dist.rvs(n_samples, random_state=random_state)

In [8]:
generate_sample(3)

array([-0.90455498, -0.15186788,  0.82050848])

In [9]:
""" Solution """

from scipy.special import logsumexp


def log_posterior_probs(x):
    """
    Computes the log posterior probabilities for the three hypotheses, given the data x

    Args:
        x (np.ndarray): one-dimensional numpy array containing the training data
    Returns:
        log_posterior_probs (np.ndarray): a numpy array of size 3, containing the Bayesian log-posterior probabilities
                                          corresponding to the three hypotheses
    """
    assert x.ndim == 1

    # Compute log-likelihood for each hypothesis
    log_likelihoods = np.array([
        np.sum(HYPOTHESIS_SPACE[0].logpdf(x)),  # Normal distribution
        np.sum(HYPOTHESIS_SPACE[1].logpdf(x)),  # Laplace distribution
        np.sum(HYPOTHESIS_SPACE[2].logpdf(x))   # Student-t distribution
    ])

    # Compute log-prior probabilities (log of PRIOR_PROBS)
    log_priors = np.log(PRIOR_PROBS)

    # Compute unnormalized log-posteriors: log P(H_i | X) = log P(X | H_i) + log P(H_i)
    log_p = log_likelihoods + log_priors

    # Normalize using logsumexp to get log P(X)
    log_p -= logsumexp(log_p)

    assert log_p.shape == (3,)
    return log_p


def posterior_probs(x):
    return np.exp(log_posterior_probs(x))

In [10]:


def main():
    """ sample from Laplace dist """
    dist = HYPOTHESIS_SPACE[1]
    x = dist.rvs(1000, random_state=28)

    print("Posterior probs for 1 sample from Laplacian")
    p = posterior_probs(x[:1])
    print("Normal: %.4f , Laplace: %.4f, Student-t: %.4f\n" % tuple(p))

    print("Posterior probs for 50 samples from Laplacian")
    p = posterior_probs(x[:50])
    print("Normal: %.4f , Laplace: %.4f, Student-t: %.4f\n" % tuple(p))

    print("Posterior probs for 1000 samples from Laplacian")
    p = posterior_probs(x[:1000])
    print("Normal: %.4f , Laplace: %.4f, Student-t: %.4f\n" % tuple(p))

    print("Posterior for 100 samples from the Bayesian data generating process")
    x = generate_sample(n_samples=100)
    p = posterior_probs(x)
    print("Normal: %.4f , Laplace: %.4f, Student-t: %.4f\n" % tuple(p))

if __name__ == "__main__":
    main()


Posterior probs for 1 sample from Laplacian
Normal: 0.3239 , Laplace: 0.2441, Student-t: 0.4320

Posterior probs for 50 samples from Laplacian
Normal: 0.2550 , Laplace: 0.3388, Student-t: 0.4062

Posterior probs for 1000 samples from Laplacian
Normal: 0.0000 , Laplace: 1.0000, Student-t: 0.0000

Posterior for 100 samples from the Bayesian data generating process
Normal: 0.0513 , Laplace: 0.0381, Student-t: 0.9106

